ETL (Extracción, Transformación y Carga) es un proceso en el que se extraen datos de una fuente, se transforman y se cargan en otra fuente. En este caso, se extraen datos de un archivo json, se transforman y se cargan en un archivo csv.

In [23]:
# Cargamos las librerias necesarias para el desarrollo del ETL.
import sys
sys.path.append("../Lib")
import utils as ut
import pandas as pd
import gzip
import ast
from dateutil import parser

Creamos una función que se encargue de leer el archivo json y devolver un dataframe de pandas.

In [24]:
# Funcion para convertir un archivo .gz a un dataframe.
def gzip_to_df(file_path):                                          
    with gzip.open(file_path, 'rt', encoding='utf-8-sig') as file:  # Abrimos el archivo .gz
        return [ast.literal_eval(line) for line in file]            # Retornamos el archivo .gz como un dataframe.

Iniciamos con el procesamientos de los datos de user_reviews.json. Para ello, hacemos uso de la función creada anteriormente y la guardamos en una variable llamada df_reviews.

In [25]:
# Cargamos los datos de los reviews y los convertimos en un dataframe.
data = gzip_to_df('../Datasets/Raw/user_Reviews.json.gz')     # Cargamos los datos de los reviews.
df_reviews = pd.DataFrame(data)                                    # Convertimos los datos de los reviews en un dataframe.
df_reviews.head()                                                  # Mostramos los primeros 5 registros del dataframe.

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [26]:
df_reviews.info() # Vemos la informacion del dataframe.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Vemos que la columna 'reviews' contiene un diccionario con los datos de cada review. Para poder acceder a los datos de cada review, creamos una función que se encargue de extraer los datos de cada review y devolver un dataframe de pandas.

In [27]:
# Desanidamos los datos de la columna reviews.
def extraer_values(df, columna):
    # Lista para almacenar los datos desanidados
    extracted_rows = []

    for index, row in df.iterrows(): # Itera sobre cada fila del DataFrame
        user_id = row["user_id"] 
        columnai = row[columna]
        
        if isinstance(columnai, list):
            for item in columnai:
                extracted_row = {"user_id": user_id}
                extracted_row.update(item)  # Agrega todas las claves y valores de 'item' al diccionario
                extracted_rows.append(extracted_row)

    # Crea un DataFrame a partir de la lista de datos desanidados
    df_new = pd.DataFrame(extracted_rows)
    return df_new

# Nombre de la columna a desanidar
columna = "reviews"

# Llama a la función con el DataFrame df_reviews
df_reviews_desanidado = extraer_values(df_reviews, columna)

df_reviews_desanidado.head() # Mostramos los primeros 5 registros del dataframe.

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


La columna 'posted' contiene la fecha en la que se publicó la review. Todas las fechas son diferentes, entonces creamos una función que se encargue de convertir la fecha a un formato más legible.

In [28]:
# Cambiamos el formato de fecha de la columna posted.
date_string = df_reviews_desanidado['posted'] # Obtiene la columna "posted"

def transform_date(date_string): # Función para transformar la fecha
    try:
        if isinstance(date_string, str):
            # Elimina "Posted" del principio y analiza la fecha
            date_string = date_string.replace('Posted', '').strip()
            parsed_date = parser.parse(date_string, fuzzy=True)
            return parsed_date
    except ValueError:
        pass  # Ignora fechas incorrectas
    return None  # Devuelve None para fechas inválidas o nulas

# Aplica la función de transformación a la columna "posted"
df_reviews_desanidado['posted'] = df_reviews_desanidado['posted'].apply(transform_date)

# Elimina las filas con valores nulos en la columna 'posted'
df_reviews_final = df_reviews_desanidado.dropna(subset=['posted'])

# Muestra el DataFrame resultante
df_reviews_final.head() 

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [29]:
# Elimina las columnas que no se utilizarán
df_reviews_final.drop(['funny', 'last_edited', 'helpful'], axis=1, inplace=True)

In [30]:
# Cuenta la cantidad de valores nulos en la columna 'posted'
cantidad_valores_nulos = df_reviews_final['posted'].isnull().sum()
print("Cantidad de valores nulos en 'posted':", cantidad_valores_nulos) 

Cantidad de valores nulos en 'posted': 0


In [31]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    59305 non-null  object        
 1   posted     59305 non-null  datetime64[ns]
 2   item_id    59305 non-null  object        
 3   recommend  59305 non-null  bool          
 4   review     59305 non-null  object        
dtypes: bool(1), datetime64[ns](1), object(3)
memory usage: 1.9+ MB


In [32]:
# Guardar el dataframe en un archivo csv en la carpeta Clean
df_reviews_final.to_csv('../Datasets/Clean/user_reviews.csv.gz', compression='gzip', index=False, encoding='utf-8')

In [33]:
# Guardar el dataframe en un archivo parquet en la carpeta Datasets
df_reviews_final.to_parquet('../Datasets/user_reviews.parquet.gz', compression='gzip', index=False)